# Session 3: Markov Chain Monte Carlo

In [1]:
import platform
import warnings

import arviz as az
import numpy as np
import plotly.graph_objects as go
import polars as pl
import pymc as pm
import scipy.stats as st
from plotly.subplots import make_subplots
from itertools import takewhile

warnings.simplefilter("ignore")

if platform.system() == 'Linux':
    # Multiprocessing behaves oddly on Linux, so we need to set the start method
    import multiprocessing as mp
    mp.set_start_method('spawn', force=True)

## Bayesian Computation

Lets take a look at [Bayes formula](https://en.wikipedia.org/wiki/Bayes%27_theorem):

$$P(\theta|x) = \frac{P(x|\theta) P(\theta)}{P(x)}$$

We have $P(\theta|x)$, the probability of our model parameters $\theta$ given the data $x$ and thus our quantity of interest. To compute this we multiply the prior $P(\theta)$ (what we think about $\theta$ before we have seen any data) and the likelihood $P(x|\theta)$, i.e. how we think our data is distributed. This nominator is pretty easy to solve for.

However, lets take a closer look at the denominator. $P(x)$ which is also called the evidence (i.e. the evidence that the data x was generated by this model). We can compute this quantity by integrating over all possible parameter values:
$$P(x) = \int_\Theta P(x, \theta) \, \mathrm{d}\theta$$

This is the key difficulty with Bayes formula -- while the formula looks innocent enough, for even slightly non-trivial models you just can't compute the posterior in closed-form. 

## Numerical Integration

Bayesian analysis often requires **integration over multiple dimensions** that is intractable both via analytic methods or standard methods of numerical integration.
However, it is often possible to compute these integrals by **simulating**
(drawing samples) from posterior distributions. For example, consider the expected value of a random variable $\mathbf{x}$:

$$E[\mathbf{x}] = \int \mathbf{x} f(\mathbf{x}) d\mathbf{x}, \qquad\mathbf{x} = x_1, \ldots ,x_k$$

where $k$ (the dimension of vector $x$) is perhaps very large. If we can produce a reasonable number of random vectors $\{{\bf x_i}\}$, we can use these values to approximate the unknown integral. This process is known as *Monte Carlo integration*. In general, MC integration allows integrals against probability density functions:

$$I = \int h(\mathbf{x}) f(\mathbf{x}) \mathbf{dx}$$

to be estimated by finite sums:

$$\hat{I} = \frac{1}{n}\sum_{i=1}^n h(\mathbf{x}_i),$$

where $\mathbf{x}_i$ is a sample from $f$. This estimate is valid and useful because:

-   By the strong law of large numbers:

$$\hat{I} \rightarrow I   \text{   with probability 1}$$

-   Simulation error can be measured and controlled:

$$Var(\hat{I}) = \frac{1}{n(n-1)}\sum_{i=1}^n (h(\mathbf{x}_i)-\hat{I})^2$$

### How is this relevant to Bayesian analysis? 

In the context of Bayesian analysis, if we replace $f(\mathbf{x})$
with a posterior, $p(\theta|y)$ and make $h(\theta)$ an interesting function of the unknown parameter, the resulting expectation is that of the posterior of $h(\theta)$:

$$E[h(\theta)|y] = \int h(\theta) p(\theta|y) d\theta \approx \frac{1}{n}\sum_{i=1}^n h(\theta)$$

## Rejection Sampling

Most integrals are hard or impossible to do. Also, if we are iterating on a statistical model, we may want a method that works without requiring rederiving a formula for generating samples. Further, in Bayesian data analysis, we may not know a *normalizing constant*: we may only know 

$$
\tilde{p}(x) = \frac{1}{Z_p}p(x),
$$

for some constant $Z_p$ ("constant" here is with respect to $x$). In order to sample, first we

1. Choose a **proposal distribution** $q$ that you know how to sample from
2. Choose a number $k$, so that $kq(x) \geq \tilde{p}(x)$ for all $x$

Then, we repeatedly 

1. Draw a $z$ from $q$
2. Draw a $u$ from $\operatorname{Uniform}(0, kq(z))$
3. If $u \leq p(x)$, accept the draw, otherwise, reject.

Importantly, every "rejection" is wasted computation! We will explore methods for having less wasted computation later.

### Example: Mixture of Gaussians

We can sample from the pdf returned by `mixture_of_gaussians` using rejection sampling. We will implement this as a Python generator, and yield the proposed draw, `z`, as well as whether it was accepted.

In [2]:
def mixture_of_gaussians():
    rvs = (st.norm(-3, 1), st.norm(0, 1), st.norm(3, 1))
    probs = (0.5, 0.2, 0.3)
    def pdf(x):
        return sum(p * rv.pdf(x) for p, rv in zip(probs, rvs))
    return pdf

In [ ]:
np.random.seed(19)
pdf = mixture_of_gaussians()
t = np.linspace(-10, 10, 500)
q = st.norm(0, 3)
z = q.rvs()
u = np.random.rand() * q.pdf(z)
k = 3

go.Figure().add_trace(
    go.Scatter(
        x=t, 
        y=[pdf(x) for x in t], 
        mode='lines', 
        name='p(x)',
        line=dict(color='blue')
    )
).add_trace(
    go.Scatter(
        x=t,
        y=[pdf(x) for x in t],
        mode='none',
        fill='tozeroy',
        fillcolor='rgba(0, 0, 255, 0.2)',
        showlegend=False
    )
).add_trace(
    go.Scatter(
        x=t, 
        y=k * q.pdf(t), 
        mode='lines', 
        name='k · N(z | 0, 3)',
        line=dict(color='orange')
    )
).add_trace(
    go.Scatter(
        x=t,
        y=k * q.pdf(t),
        mode='none',
        fill='tonexty',
        fillcolor='rgba(255, 165, 0, 0.2)',
        showlegend=False
    )
).add_shape(
    type="line",
    x0=z, x1=z,
    y0=0, y1=pdf(z),
    line=dict(color="green", dash="dash")
).add_shape(
    type="line",
    x0=z, x1=z,
    y0=pdf(z), y1=k * q.pdf(z),
    line=dict(color="red", dash="dash"),
).add_trace(
    go.Scatter(
        x=[z], 
        y=[pdf(z)], 
        mode='markers', 
        marker=dict(size=15, color='blue', line=dict(width=2, color='white')),
        name='p(z)',
        showlegend=False
    )
).add_trace(
    go.Scatter(
        x=[z], 
        y=[u], 
        mode='markers', 
        marker=dict(size=9, symbol='star', color='green'),
        name='u ~ U(0, k·N(z | 0, 3))'
    )
).add_trace(
    go.Scatter(
        x=[z], 
        y=[k * q.pdf(z)], 
        mode='markers', 
        marker=dict(size=15, color='orange', line=dict(width=2, color='white')),
        name='k·q(z)',
        showlegend=False
    )
).update_layout(
    width=750, 
    height=375,
    xaxis=dict(range=[-10, 10]),
    yaxis=dict(range=[0, 0.45]),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

> #### What is a Generator?
> A Python generator is a special type of iterator that allows you to iterate over a sequence of values lazily, meaning it generates values on-the-fly and only when requested, rather than storing the entire sequence in memory. This is accomplished using the `yield` statement within a function, which produces a value and pauses the function's execution, resuming from that point when the next value is requested. Thus, instead of generating and storing all values at once, which can be resource-intensive, generators produce values one at a time, thus reducing memory overhead and allowing for the handling of infinite sequences or very large data sets that would be impractical to store in memory. 

Below we have a function that uses a generator to `yield` samples from a given target distribution and proposal, and a second function that orchestrates the sampling process.

In [4]:
def rejection_sampler(pdf, proposal_dist, k):
    while True:
        # Generate proposal
        z = proposal_dist.rvs()
        q_dist = proposal_dist.pdf(z)
        # Draw uniformly from 0 to k * q(z)
        u = k * np.random.rand() * q_dist
        # Check our envelope condition
        assert k * q_dist >= pdf(z)
        if u <= pdf(z):
            accept = True
        else:
            accept = False
        yield z, accept

def gen_samples(draws, sampler):
    samples = []
    for n_draws, (z, accept) in enumerate(sampler, start=1):
        if accept:
            samples.append(z)
            if len(samples) == draws:
                return np.array(samples), n_draws

Notice that if $kq(x)$ is not larger than $\tilde{p}(x)$, an exception is thrown.

Let's try to recover the mixture distribution with rejection sampling. We will use a normal distribution as our proposal distribution. 

In [5]:
pdf = mixture_of_gaussians()
proposal_dist = st.norm(0, 3)
k = 4
N = 10_000

samples, draws = gen_samples(N, rejection_sampler(pdf, proposal_dist, k))

Let's visualize the results:

In [ ]:
t = np.linspace(samples.min(), samples.max(), 500)
pdf_values = [pdf(x) for x in t]

hist = go.Histogram(
    x=samples,
    histnorm='probability density',
    name='Samples',
    marker=dict(color='rgba(0, 0, 255, 0.7)'),
    autobinx=True
)

pdf_curve = go.Scatter(
    x=t,
    y=pdf_values,
    mode='lines',
    name='True PDF',
    line=dict(color='orange', width=2)
)

go.Figure(
    data=[hist, pdf_curve]
).update_layout(
    title=f'{samples.size:,d} draws from the pdf with {100 * samples.size / draws:.2f}% efficiency',
    xaxis_title='Value',
    yaxis_title='Density',
    width=750,
    height=525,
    bargap=0.01,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

### How does a rejection sampler scale with dimension?

Let's use a multivariate Gaussian with identity covariance matrix as a target distribution, and a multivariate Gaussian with covariance matrix `1.1 * I` as the proposal. Should be easy, right? 

- Around what percent of samples are accepted with dimension 1? 
- 10 dimensions? 
- 100 dimensions? 
- What happens if you try to use 1,000 dimensions?

In [ ]:
DIM = 1

def finite_sampler(attempts, sampler):
    samples = []
    for n_draws, (z, accept) in takewhile(lambda j: j[0] < attempts, enumerate(sampler)):
        if accept:
            samples.append(z)
    return np.array(samples)        

pdf = st.multivariate_normal(np.zeros(DIM), np.eye(DIM)).pdf
proposal_dist = st.multivariate_normal(np.zeros(DIM), 1.1 * np.eye(DIM))
k = pdf(0) / proposal_dist.pdf(0)

sampler = rejection_sampler(pdf, proposal_dist, k)

samples = finite_sampler(1_000, sampler)

print(f"Acceptance rate: {len(samples) / 1000:.3f}")

## Introduction to MCMC

One way to intuitively waste less computation is to use knowledge from your current sample to inform your next proposal: this is called a **Markov chain**. 


> ## Markov Chains
>
> A Markov chain is a special type of *stochastic process*. The standard definition of a stochastic process is an ordered collection of random variables:
>
> $$\{X_t: t \in T\}$$
> 
> where $t$ is frequently (but not necessarily) a time index. If we think of $X_t$ as a state $X$ at time $t$, and invoke the following dependence condition on each state:
> 
> $$Pr(X_{t+1}=x_{t+1} | X_t=x_t, X_{t-1}=x_{t-1},\ldots,X_0=x_0) = Pr(X_{t+1}=x_{t+1} | X_t=x_t)$$
> 
> then the stochastic process is known as a Markov chain. This conditioning specifies that the future depends on the current state, but not past states.



Let $t$ be the index of our current sample, $x_t$ be our current sample, and $\operatorname{pdf}(x_t)$ be our probability density function evaluated at the current sample. We will define a *transition probability* that is conditioned on our current position: $T(x_{t + 1} | x_t)$. 

Critically, a Markov chain will sample from $\operatorname{pdf}$ if:

- $T$ is ergodic (this means $T$ is aperiodic and can explore the whole space)
- The chain satisfies the *detailed balance* condition, which says that $\operatorname{pdf}(x_t)T(x_{t+1} | x_t) = \operatorname{pdf}(x_{t + 1})T(x_{t} | x_{t + 1})$.

This second criteria inspires the *Metropolis acceptance criterion*: If we use any proposal with density function $\operatorname{prop}$, we use this criterion to "correct" the transition probability to satisfy detailed balance:

$$
A(x_{t + 1} | x_t) = \min\left\{1, \frac{\operatorname{pdf}(x_{t + 1})}{\operatorname{pdf}(x_{t})}\frac{\operatorname{prop}(x_{t} | x_{t + 1})}{\operatorname{prop}(x_{t + 1} | x_t)} \right\}
$$

Now the *Metropolis-Hastings Algorithm* is

Initialize at some point $x_0$. For each iteration:

1. Draw $\tilde{x}_{t + 1} \sim \operatorname{prop}(x_t)$
2. Draw $u \sim \operatorname{Uniform}(0, 1)$
3. If $u < A(\tilde{x}_{t + 1} | x_t)$, then $x_{t + 1} = \tilde{x}_{t + 1}$. Otherwise, $x_{t + 1} = x_t$.

Similar to the rejection sampler, below is a function to generate Metropolis-Hastings samples and a second function to orchestrate the sampling process.

In [8]:
def metropolis_hastings(pdf, prop_dist, init=0):
    """Yields a sample, and whether it was accepted."""
    current = init
    while True:
        prop = prop_dist.rvs()
        p_accept = min(1, pdf(prop) / pdf(current) * prop_dist.pdf(current) / prop_dist.pdf(prop))
        accept = np.random.rand() < p_accept
        if accept:
            current = prop
        yield current, accept
        
def gen_samples(draws, sampler):
    """An example of using the metropolis_hastings API."""
    samples = np.empty(draws)
    accepts = 0
    for idx, (z, accept) in takewhile(lambda j: j[0] < draws, enumerate(sampler)):
        accepts += int(accept)
        samples[idx] = z
    return samples, accepts


Let's try this on the mixture of Gaussians example. Again, we will use a normal distribution as our proposal distribution.

In [9]:
pdf = mixture_of_gaussians()
proposal_dist = st.norm(0, 10)

samples, accepts = gen_samples(10_000, metropolis_hastings(pdf, proposal_dist))

In [ ]:
t = np.linspace(samples.min(), samples.max(), 500)
pdf_values = [pdf(x) for x in t]

hist = go.Histogram(
    x=samples,
    histnorm='probability density',
    name='Samples',
    marker=dict(color='rgba(0, 0, 255, 0.7)'),
    nbinsx=30
)

pdf_curve = go.Scatter(
    x=t,
    y=pdf_values,
    mode='lines',
    name='True PDF',
    line=dict(color='orange', width=2)
)

go.Figure(
    data=[hist, pdf_curve]
).update_layout(
    title=f'{samples.size:,d} draws from the pdf with {100 * accepts / samples.size:.2f}% accept rate',
    xaxis_title='Value',
    yaxis_title='Density',
    width=750,
    height=525,
    bargap=0.01,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

Pretty good, but implementation above is rather inefficient! 

We will speed it up by fixing the proposal distribution as a Gaussian **centered** at the previous point. Specifically,
$$x_{t+1} \sim \mathcal{N}( x_t, \sigma),$$
so
$$\operatorname{prop}(x_{t+1} | x_{t}) = \mathcal{N}(x_{t + 1} | x_t, \sigma)$$

This is a *random walk* proposal, where $\sigma$ is the **step size**.

### Random Walk Metropolis-Hastings

A common special case of Metropolis-Hastings is random walk Metropolis-Hastings. In random walk MCMC, the proposal distribution is symmetric around the current position. This simplifies the acceptance ratio calculation because the ratio of the proposal distributions cancels out:

$$A(x_{t + 1} | x_t) = \min\left\{1, \frac{\operatorname{pdf}(x_{t + 1})}{\operatorname{pdf}(x_{t})}\right\}$$

This is actually both simpler to implement and more efficient for generating samples.

In [11]:
def random_walk_metropolis(pdf, step_size, init=0):
    """Random walk Metropolis algorithm"""
    current = init
    while True:
        # Random walk proposal
        prop = current + np.random.normal(0, step_size)
        # Simple acceptance ratio (proposal terms cancel out)
        p_accept = min(1, pdf(prop) / pdf(current))
        accept = np.random.rand() < p_accept
        if accept:
            current = prop
        yield current, accept

The critical hyperparameter in random walk Metropolis-Hastings is the step size, $\sigma$. If $\sigma$ is too small, the chain will take small steps and converge slowly. If $\sigma$ is too large, the chain will reject many proposals and converge slowly.


The optimal step size balances the **exploration** of a large step size and the **exploitation** of a small step size. A common rule of thumb is to set the acceptance rate to be around 1/4.

In [ ]:
pdf = mixture_of_gaussians()

# Create subplots
fig = make_subplots(rows=3, cols=1, 
                   subplot_titles=["Step size: 0.1", "Step size: 1.0", "Step size: 5.0"],
                   vertical_spacing=0.1,
                   specs=[[{"type": "xy"}], [{"type": "xy"}], [{"type": "xy"}]])

# Try different step sizes
step_sizes = [0.1, 8.0, 70.0]

for i, step_size in enumerate(step_sizes, 1):
    # Generate samples
    samples, accepts = gen_samples(10_000, random_walk_metropolis(pdf, step_size))
    
    # Calculate t and pdf values for the line
    t = np.linspace(samples.min(), samples.max(), 500)
    pdf_values = [pdf(x) for x in t]
    
    # Add histogram
    fig.add_trace(
        go.Histogram(
            x=samples,
            histnorm='probability density',
            marker=dict(color='rgba(0, 0, 255, 0.7)'),
            name=f"Samples (step={step_size})",
            showlegend=False
        ),
        row=i, col=1
    )
    
    # Add PDF line
    fig.add_trace(
        go.Scatter(
            x=t,
            y=pdf_values,
            mode='lines',
            line=dict(color='orange', width=2),
            name=f"True PDF (step={step_size})",
            showlegend=False
        ),
        row=i, col=1
    )
    
    # Update subplot title to include accept rate
    fig.layout.annotations[i-1].text = f"Step size: {step_size}, Accept rate: {100 * accepts / samples.size:.2f}%"

# Update layout
fig.update_layout(
    height=750,
    width=750,
    bargap=0.01,
    showlegend=False
)


fig.update_xaxes(title_text="Value", row=3, col=1)
fig.update_yaxes(title_text="Density", row=2, col=1)

fig.show()


## Hamiltonian Monte Carlo

While flexible and easy to implement, Metropolis-Hastings sampling is a random walk
sampler that might not be statistically efficient for many models. Specifically, for models of **high dimension**, random walk jumping algorithms do not perform well. It is not enough to simply guess at the next sample location; we need to make each iteration a viable draw from the posterior whenever we can, in order to have an efficient sampler for bigger models.

Since Bayesian inference is all about calculating expectations over posteriors, what we seek is an algorithm that explores the area of the parameter space that contains most of the non-zero probability. This region is called the **typical set**.

### What's a Typical Set?

The typical set is where most of the **probability** lies in a particular volume associated with the distribution. As the dimension of a model increases, this set moves progressively further from the mode, and becomes more singular, as the result of concentration of measure.

The typical set is a product of both the **density**, which is highest at the mode, and **volume** (that we integrate over), which increasingly becomes larger away from the mode as dimensionality increases. In fact, at high dimensions, the region around the mode contributes almost nothing to the expectation. Random walk methods have trouble finding and traversing this set efficiently. We need an algorithm that will find this narrow region and explore it efficiently.

![from Betancourt 2018](images/typicalset.png)

In this context, and when sampling from continuous variables, **Hamiltonian Monte
Carlo (HMC)** can prove to be a powerful tool. It avoids
random walk behavior by **simulating a physical system** governed by
Hamiltonian dynamics, potentially avoiding tricky conditional
distributions in the process.

HMC introduces two key concepts:

1. **Momentum variables**: In addition to position variables (our parameters of interest), HMC introduces momentum variables, creating a phase space.
2. **Hamiltonian dynamics**: It uses physical dynamics to guide proposals along trajectories that maintain constant energy.

In HMC, model samples are obtained by simulating a physical system,
where particles move about a high-dimensional landscape, subject to
potential and kinetic energies. Adapting the notation from [Neal (1993)](http://www.cs.toronto.edu/~radford/review.abstract.html),
particles are characterized by a position vector or state
$s \in \mathcal{R}^D$ and velocity vector $\phi \in \mathcal{R}^D$. The
combined state of a particle is denoted as $\chi=(s,\phi)$. 

The joint **canonical distribution** of the position and velocity can be expressed as a product of the marginal position (which is of interest) and the conditional distribution of the velocity:

$$\pi(s, \phi) = \pi(\phi | s) \pi(s)$$

This joint probability can also be written in terms of an invariant **Hamiltonian function**:

$$\pi(s, \phi) \propto \exp(-H(s,\phi))$$

The Hamiltonian is then defined as the sum of potential energy $E(s)$ and kinetic energy
$K(\phi)$, as follows:

$$\mathcal{H}(s,\phi) = E(s) + K(\phi)
= E(s) + \frac{1}{2} \sum_i \phi_i^2$$

Instead of sampling $p(s)$ directly, HMC operates by sampling from the canonical distribution.

$$p(s,\phi) = \frac{1}{Z} \exp(-\mathcal{H}(s,\phi))=p(s)p(\phi)$$

If we choose a momentum that is independent of position, marginalizing over $\phi$ is
trivial and recovers the original distribution of interest.

Note that the Hamiltonian $\mathcal{H}$ is independent of the parameterization of the model, and therefore, captures the geometry of the phase space distribution, including typical set. 

**Hamiltonian Dynamics**

State $s$ and velocity $\phi$ are modified such that
$\mathcal{H}(s,\phi)$ remains constant throughout the simulation. The
differential equations are given by:

$$\begin{aligned}\frac{ds_i}{dt} &= \frac{\partial \mathcal{H}}{\partial \phi_i} = \phi_i \\
\frac{d\phi_i}{dt} &= - \frac{\partial \mathcal{H}}{\partial s_i}
= - \frac{\partial E}{\partial s_i}
\end{aligned}$$

As shown in [Neal (1993)](http://www.cs.toronto.edu/~radford/review.abstract.html), 
the above transformation **preserves volume** and is
**reversible** (*i.e.* satisfies detailed balance). The above dynamics can thus be used as transition operators
of a Markov chain and will leave $p(s,\phi)$ invariant. That chain by
itself is not ergodic however, since simulating the dynamics maintains a
fixed Hamiltonian $\mathcal{H}(s,\phi)$. HMC thus **alternates** Hamiltonian
dynamic steps, with Gibbs sampling of the velocity. Because $p(s)$ and
$p(\phi)$ are independent, sampling $\phi_{new} \sim p(\phi|s)$ is
trivial since $p(\phi|s)=p(\phi)$, where $p(\phi)$ is often taken to be
the univariate Gaussian.

![Skate park](images/skate_park.png?raw=true)

**The Leap-Frog Algorithm**

In practice, we cannot simulate Hamiltonian dynamics exactly because of
the problem of time discretization. There are several ways one can do
this. To maintain invariance of the Markov chain however, care must be
taken to preserve the properties of *volume conservation* and *time
reversibility*. The **leap-frog algorithm** maintains these properties
and operates in 3 steps:

$$\begin{aligned}
\phi_i(t + \epsilon/2) &= \phi_i(t) - \frac{\epsilon}{2} \frac{\partial{}}{\partial s_i} E(s(t)) \\
s_i(t + \epsilon) &= s_i(t) + \epsilon \phi_i(t + \epsilon/2) \\
\phi_i(t + \epsilon) &= \phi_i(t + \epsilon/2) - \frac{\epsilon}{2} \frac{\partial{}}{\partial s_i} E(s(t + \epsilon)) 
\end{aligned}$$

We thus perform a half-step update of the velocity at time
$t+\epsilon/2$, which is then used to compute $s(t + \epsilon)$ and
$\phi(t + \epsilon)$.

**Accept / Reject**

In practice, using finite stepsizes $\epsilon$ will not preserve
$\mathcal{H}(s,\phi)$ exactly and will introduce bias in the simulation.
Also, rounding errors due to the use of floating point numbers means
that the above transformation will not be perfectly reversible.

HMC cancels these effects **exactly** by adding a Metropolis
accept/reject stage, after $n$ leapfrog steps. The new state
$\chi' = (s',\phi')$ is accepted with probability $p_{acc}(\chi,\chi')$,
defined as:

$$p_{acc}(\chi,\chi') = min \left( 1, \frac{\exp(-\mathcal{H}(s',\phi')}{\exp(-\mathcal{H}(s,\phi)} \right)$$

**HMC Algorithm**

We obtain a new HMC sample as follows:

1.  sample a new velocity from a univariate Gaussian distribution
2.  perform $n$ leapfrog steps to obtain the new state $\chi'$
3.  perform accept/reject move of $\chi'$

### No-U-Turn Sampler (NUTS)

A key challenge with HMC is selecting appropriate values for the step size $\epsilon$ and number of steps $L$. The No-U-Turn Sampler (NUTS) addresses this by:

1. **Adaptively tuning the step size** during a warm-up phase to achieve a target acceptance rate
2. **Dynamically determining the number of steps** by running the leapfrog integrator until a U-turn occurs (when the trajectory starts moving back toward its starting point)

NUTS builds a binary tree of leapfrog steps and stops when the trajectory makes a U-turn. For each proposal, it starts with a single leapfrog step.
It then doubles the length of the trajectory, simulating Hamiltonian dynamics both forwards and backwards in "time" from the current endpoints. This creates a balanced binary tree of states. It continues extending the trajectory by doubling the number of steps, subject to several conditions (related to detailed balance and ensuring good exploration). 
Once the doubling process stops (e.g., a U-turn is detected or a maximum tree depth is reached), a point is randomly sampled from all the states in the generated trajectory, with probabilities that ensure the overall sampler is valid.

<img src="images/uturn.png" alt="No-U-Turn Sampler illustration" width="50%">

PyMC implements NUTS as its default sampler because of its efficiency for a wide range of problems. Let's see how to use it in practice:

## MCMC in PyMC

PyMC's core business is using Markov chain Monte Carlo to fit virtually any probability model. This involves the assignment and coordination of a suite of **step methods**, each of which is responsible for updating one or more variables. 

The user's interface to PyMC's sampling algorithms is the `sample` function, which coordinates the sampling process for a model.

In the previous session, we specified a model for estimating basketball team strength based on game outcomes. Let's now try and fit this model.

In [13]:
team_data = pl.read_parquet('../data/ncaa_team_data.parquet')
game_data = pl.read_parquet("../data/ncaa_game_data.parquet")

predictor_cols = ['FG%', '3P%', 'FT%', 'ORB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF']

# Standardize the predictor columns in polars
X = team_data.select(
    [(pl.col(c) - pl.col(c).mean()) / pl.col(c).std() for c in predictor_cols]
)

y = game_data['home_margin'].to_numpy()

coords = dict(
    predictor=predictor_cols,
    team=team_data["School"].to_list()
)

with pm.Model(coords=coords) as ncaa_model:
    
    # Predictor coefficients
    beta = pm.Normal('beta', 0, sigma=10, dims="predictor")

    # Home advantage
    home_advantage = pm.HalfNormal('home_advantage', sigma=10)

    # Observation error
    sigma = pm.HalfCauchy('sigma', 1)

    team_strength = pm.Deterministic('team_strength', beta.dot(X.to_numpy().T), dims="team")

    expected_margin = home_advantage + team_strength[game_data['home_team_id'].to_numpy()] - team_strength[game_data['away_team_id'].to_numpy()]

    pm.Normal('outcome', expected_margin, sigma=sigma, observed=y)

Let's first see what happens when you run `pm.sample` with no arguments:

In [ ]:
with ncaa_model:
    idata = pm.sample()

### Supported sampling methods

PyMC offers a range of sampling methods to accommodate different model requirements:

1. **NUTS (No U-Turn Sampler)** - The default for continuous variables, an advanced HMC variant that automatically tunes parameters

2. **HamiltonianMC** - Basic Hamiltonian Monte Carlo implementation for continuous variables

3. **Metropolis** - Classic Metropolis-Hastings algorithm that can handle continuous and discrete variables

4. **BinaryMetropolis** - Specialized algorithm for binary (boolean) variables

5. **Slice** - Slice sampling algorithm

6. **SGMC (Stochastic Gradient MCMC)** - Methods that use gradient information for large datasets

The `sample` function generally selects the appropriate method automatically based on the variable types in your model, though you can manually specify methods if needed.There is rarely a reason to use `HamiltonianMC` rather than `NUTS`. It is the default sampler for continuous variables in PyMC.

### Automatic assignment of step methods

When `step` is not specified by the user, PyMC will assign step methods to variables automatically. To do so, each step method implements a class method called `Competence`. This method returns a value from 0 (incompatible) to 3 (ideal), based on the attributes of the random variable in question. `sample()` assigns the step method that returns the highest competence value to each of its unallocated stochastic random variables. In general:

* Binary variables will be assigned to `BinaryMetropolis` (Metropolis-Hastings for binary values)
* Discrete variables will be assigned to `Metropolis`
* Continuous variables will be assigned to `NUTS` (No U-turn Sampler)

### Parallel sampling

Nearly all modern desktop computers have multiple CPU cores, and running multiple MCMC chains is an **embarrasingly parallel** computing task. It is therefore relatively simple to run chains in parallel in PyMC. This is done by setting the `cores` argument in `sample` to some value between 2 and the number of cores on your machine (you can specify more chains than cores, but you will not gain efficiency by doing so). The default value of `cores` is `None`, which will select the number of CPUs on your machine, to a maximum of 4. 

> Keep in mind that some chains might themselves be multithreaded via openmp or BLAS. In those cases it might be faster to set this to 1.

However, note that the number of chains sampled can be set independently of the number of cores by specifying the `chains` argument.

```python
with my_model:
    trace = pm.sample(chains=4, cores=2)
```

Running $n$ iterations with $c$ chains will result in $c \times n$ samples.

Generating several chains is generally recommended because it aids in model checking, allowing statistics such as the potential scale reduction factor ($\hat{R}$) and effective sample size to be calculated.

In [ ]:
with ncaa_model:
    # Sample 2 chains in parallel using 2 cores
    trace = pm.sample(1000, tune=1000, chains=2, cores=2)

## JAX-based Samplers

An alternative to PyMC's PyTensor-based samplers are samplers written in JAX. Using these samplers, all the operations needed to compute a posterior can be performed under JAX, reducing the Python overhead during sampling and leveraging all JAX performance improvements and features like the ability to sample on GPUs or TPUs.

PyMC offers NUTS JAX samplers via [NumPyro](https://num.pyro.ai/en/latest/index.html) or [Nutpie](https://github.com/pymc-devs/nutpie). Significantly, NumPyro and Nutpie can both be used because in PyMC the modeling language is decoupled from the inference methods; NumPyro and Nutpie only require a log-probability density function written in JAX. This demonstrates that samplers can be developed independently of PyMC and then be made available to users of the library.

The JAX samplers can be invoked using the `nuts_sampler` argument for `pm.sample`, either using the Numpyro backend or the Nutpie backend:

```python
pm.sample(nuts_sampler="numpyro")
```

or

```python
pm.sample(nuts_sampler="nutpie", nuts_sampler_kwargs={"backend": "jax"})
```

In [ ]:
with ncaa_model:
    # Use the Nutpie JAX sampler
    trace = pm.sample(nuts_sampler="numpyro", nuts_sampler_kwargs={"chain_method": "parallel"})

---
## References

Betancourt, M. (2018). A Conceptual Introduction to Hamiltonian Monte Carlo. arXiv:1209.3572v1 [stat.CO].

Doucet, A., De Freitas, N., and Gordon, N. (2001), Sequential Monte Carlo Methods in Practice, Statistics for Engineering and Information Science, New York: Springer-Verlag.

Chapter 6 of [Givens, Geof H.; Hoeting, Jennifer A. (2012-10-09). Computational Statistics (Wiley Series in Computational Statistics)](http://www.stat.colostate.edu/computationalstatistics/)

Chapter 5 of [Albert, J. (2009). Bayesian computation with R.](http://www.amazon.com/Bayesian-Computation-R-Use/dp/0387922970)

Gelman, A., Carlin, J. B., Stern, H. S., & Rubin, D. B. (2003). Bayesian Data Analysis, Second Edition (Chapman & Hall/CRC Texts in Statistical Science) (2nd ed.). Chapman and Hall/CRC.

Neal, R. M. (2003). Slice sampling. The Annals of Statistics, 31(3), 705–767. doi:10.1111/1467-9868.00198

In [ ]:
%load_ext watermark
%watermark -n -u -v -iv -w